<a href="https://colab.research.google.com/github/eliconger/eliconger/blob/main/BNFO301_2025_Lab7_3_Student2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Lab 7. Phylogenetic tree inference using distance matrix methods.

In this three part lab, you will infer a phylogenetic tree using a Distance Matrix Method. In order to infer a tree using this method, we must take several steps, starting with sequences that have been aligned (i.e. they are of equal length), we must find all pairwise distances and correct the distances using one of many models of subsitution, group sequence pairs and re-calculate distances and repeat these processes until all pairs have been grouped.  This order of grouping determines the order taxa in the tree, which will be represented as a newick string.

Reminder: **Do not use AI when completing this assignment.** While you may discuss the lab with other students, you may not copy another students code.

In order to get started, run the boxes with the provided code in order to compile the required functions you will need today.

##Part 3. 10 points - April 3

Problem 3.1 create a function to infer a phylogenetic tree

Problem 3.2 create a function to read in a sequence alignment in phylip format

Problem 3.3 use functions created in parts 1 and 2 to infer a phylogenetic tree given the alignment data provided in a phylip file

Problem 3.3 Answer questions about this tree

In [ ]:
import math

Box 1.1, function used to calculate a P-distance given two sequences

In [ ]:
#####################

def PDistance(x, y):
	assert len(x) == len(y), 'sequences must be of equal length'
	counter = 0.0
	for i in range(len(x)):
		if (x[i] != y[i]):
			counter += 1
	Pdist = counter/len(x)
	return Pdist

#####################
#
#use these two sequences to test your function;
#P-distance = 0.2
#
######################

seq1a = "TATAGTAAAA"
seq2a = "TATAGTATTA"
Dist_Problem1_1 = PDistance(seq1a, seq2a)
print(Dist_Problem1_1)

0.2


Box 1.2, fucntion used to correct a P-distance using the Jukes-Cantor model

In [ ]:
#####################

def JCdistance(x, y):
  uncorrectedDist = PDistance(x, y)
  if (uncorrectedDist != 0):
    correctdist = (-3/4)*math.log(1-(4/3)*uncorrectedDist)
  else:
    correctdist = 0.0
  return (correctdist)

#####################
#
#use these two sequences to test your function;
#Corrected P-distance = 0.2326161962278796
#
######################

seq1b = "TATAGTTAAA"
seq2b = "TATAGTTTTA"
JCdist_Problem1_2 = JCdistance(seq1b, seq2b)
print(JCdist_Problem1_2)

0.2326161962278796


Box 1.3, a function used to create a ditional containing all pairwise corrected distances.  



In [ ]:
#####################

def DistanceMatrix(sequences):
	matrix = {}
	for i in sequences.keys():
		if i not in matrix.keys():
			matrix[i] = {}
		for j in sequences.keys():
			matrix[i][j] = JCdistance(sequences[i], sequences[j])

	return matrix

#####################
#
#use these sequences to test your function
#expected result: {'seq1': {'seq1': 0.0, 'seq2': 0.2326161962278796, 'seq3': 0.3831192178244929, 'seq4': 0.2326161962278796}, 'seq2': {'seq1': 0.2326161962278796, 'seq2': 0.0, 'seq3': 0.3831192178244929, 'seq4': 0.0}, 'seq3': {'seq1': 0.3831192178244929, 'seq2': 0.3831192178244929, 'seq3': 0.0, 'seq4': 0.3831192178244929}, 'seq4': {'seq1': 0.2326161962278796, 'seq2': 0.0, 'seq3': 0.3831192178244929, 'seq4': 0.0}}
#
#####################

SampleSeqDict = {'seq1': 'TATACTAAAA', 'seq2': 'TATACTAATT', 'seq3': 'TATAGTATTA', 'seq4': 'TATACTAATT'}
Matrix_Problem1_3 = DistanceMatrix(SampleSeqDict)
print(Matrix_Problem1_3)

{'seq1': {'seq1': 0.0, 'seq2': 0.2326161962278796, 'seq3': 0.3831192178244929, 'seq4': 0.2326161962278796}, 'seq2': {'seq1': 0.2326161962278796, 'seq2': 0.0, 'seq3': 0.3831192178244929, 'seq4': 0.0}, 'seq3': {'seq1': 0.3831192178244929, 'seq2': 0.3831192178244929, 'seq3': 0.0, 'seq4': 0.3831192178244929}, 'seq4': {'seq1': 0.2326161962278796, 'seq2': 0.0, 'seq3': 0.3831192178244929, 'seq4': 0.0}}


Box 1.4, a function used to find the most similar sequences, given the dictionary created in the last function.

In [ ]:
#####################

def GetMinDistance(matrix):
	lowest_val = float("inf")
	lowest_i = ""
	lowest_j = ""

	for i in matrix.keys():
		for j in matrix.keys():
			if (matrix[i][j] < lowest_val) and (i != j):
				lowest_val = matrix[i][j]
				lowest_i = i
				lowest_j = j

	return lowest_i, lowest_j

#####################
#
#use these sequences to test your function, seq1 and seq2 are most similar within the test data
#Expected result: seq1 and seq2
#
#####################

SampleSeqDict = {'seq1': 'TATAGTATTA', 'seq2': 'TATAGTATTA', 'seq3': 'AATAGTATTA', 'seq4': 'TATACAATTA'}
Matrix_Problem1_4 = DistanceMatrix(SampleSeqDict)
i, j = GetMinDistance(Matrix_Problem1_4)
print(i,j)

seq1 seq2


Box 2.1, a function used to merge the closest neighbors and shrink the nested dictionary created in the previous function.

In [ ]:
#####################
def ShrinkMatrix(matrix):
	name1,name2 = GetMinDistance(matrix)
	newLabel = "({0},{1})".format(name1, name2)
	namelist = list(set(matrix.keys()).difference(set([name1, name2])))
	nMatrix = {}
	for first in namelist:
		if first not in nMatrix.keys():
			nMatrix[first] = {}
		for second in namelist:
			nMatrix[first][second] = matrix[first][second]
	nMatrix[newLabel] = {}
	for other in namelist:
		d1 = matrix[other][name1]
		d2 = matrix[other][name2]
		avg = (d1 + d2) / 2.0
		nMatrix[newLabel][other] = avg
		nMatrix[other][newLabel] = avg
	nMatrix[newLabel][newLabel] = 0.0
	return nMatrix


#####################
#
#use these sequences to test your function,
#Expected result: 'seq1' and 'seq2' should be grouped in the new dictionary
#
#####################

SeqDict_Problem2_1 = {'seq1': 'TATAGTATTA', 'seq2': 'TATAGTATTA', 'seq3': 'AATAGTATTA', 'seq4': 'TATACAATTA'}
Distances_Problem2_1 = DistanceMatrix(SeqDict_Problem2_1)
Matrix_Problem2_1 = ShrinkMatrix(Distances_Problem2_1)
print(Distances_Problem2_1)
print(Matrix_Problem2_1)

{'seq1': {'seq1': 0.0, 'seq2': 0.0, 'seq3': 0.10732563273050497, 'seq4': 0.2326161962278796}, 'seq2': {'seq1': 0.0, 'seq2': 0.0, 'seq3': 0.10732563273050497, 'seq4': 0.2326161962278796}, 'seq3': {'seq1': 0.10732563273050497, 'seq2': 0.10732563273050497, 'seq3': 0.0, 'seq4': 0.3831192178244929}, 'seq4': {'seq1': 0.2326161962278796, 'seq2': 0.2326161962278796, 'seq3': 0.3831192178244929, 'seq4': 0.0}}
{'seq4': {'seq4': 0.0, 'seq3': 0.3831192178244929, '(seq1,seq2)': 0.2326161962278796}, 'seq3': {'seq4': 0.3831192178244929, 'seq3': 0.0, '(seq1,seq2)': 0.10732563273050497}, '(seq1,seq2)': {'seq4': 0.2326161962278796, 'seq3': 0.10732563273050497, '(seq1,seq2)': 0.0}}


Problem 3.1. Create a function to infer a phylogenetic tree using corrected pairwise distances. The function should expect a dictionary and use "SrinkMatrix" fuction to reduce the dictionary until only one key remains. The final key will be your newick tree string. Pleae return the newick string.  Please note, this function should only contain 3 to 4 lines of code.

In [ ]:
#####################
#
#create a definition called "ConstructPhylogeneticTree" that will return a newick string
#
#####################
def ConstructPhylogeneticTree(distance_matrix): #~
  # for all columns in the cell
    # call shrinkmatrix on new matrix
  new_distance_matrix = distance_matrix
  for i in range(len(distance_matrix)-1):
    new_distance_matrix = ShrinkMatrix(new_distance_matrix)

  remaining_key = next(iter(new_distance_matrix))
  return remaining_key


    ### matrix returned first by shrinkwill be matrix with newick string




#####################
#
#use these sequences to test your function
#Expected result: (seq4,(seq3,(seq1,seq2)));
#
#####################

SeqDictProblem3_1 = {'seq1': 'TATAGTATTA', 'seq2': 'TATAGTATTA', 'seq3': 'AATAGTATTA', 'seq4': 'TATACAATTA'}
DistanceProblem3_1 = DistanceMatrix(SeqDictProblem3_1)
newickStringProblem3_1 = ConstructPhylogeneticTree(DistanceProblem3_1)
print(newickStringProblem3_1)

(seq4,(seq3,(seq1,seq2)))


Please run box below to load a phylip formated file containing real sequence data. You will use this data to test your code in the problems below.

In [ ]:
import os.path
# Load the phylip file
DATA_FILE_GITHUB = "https://raw.githubusercontent.com/boydvcu/BNFO301_2023/main/NADH2_Shark.phy"
DEFAULT_FILE_NAME = 'NADH2_Shark.phy'

fileName = DEFAULT_FILE_NAME
#Does the file exists locally, if not get it from the github
if not os.path.exists(fileName):
  #Load the file from Github to the local folder
  !wget --no-check-certificate --content-disposition $DATA_FILE_GITHUB

print("Reading file:", fileName)

# Read in the string from the file
with open(fileName, "r") as myfile:
    file_input = myfile.read()

--2025-04-04 17:26:55--  https://raw.githubusercontent.com/boydvcu/BNFO301_2023/main/NADH2_Shark.phy
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32765 (32K) [text/plain]
Saving to: ‘NADH2_Shark.phy’

NADH2_Shark.phy     100%[===================>]  32.00K  --.-KB/s    in 0.05s   

2025-04-04 17:26:55 (670 KB/s) - ‘NADH2_Shark.phy’ saved [32765/32765]

Reading file: NADH2_Shark.phy


Problem 3.2. In order to prepare the data for phylogenetic inference, create a function to read in sequence data obtain in the box above.  Store this data in a dictionary where the sequence identifier is the key and the sequence itself if the value.

In [34]:
#####################
#
#create a definition called LoadPhylip that will load the real data and create a ditionary of sequence name-sequence key-pairs
#
#####################
def LoadPhylip(filepath):
  sequences = {}
  current_header = None
  with open(filepath, 'r') as file:
    for line in file:
      line = line.strip()
      if '.1' in line:
        # split line into label and sequence
        parts = line.split()
        current_header = parts[0].strip('.1')
        sequences[current_header] = ''
        if len(parts) > 1:
          sequences[current_header] += parts[1]
      elif current_header:
        sequences[current_header] += line
    return sequences



#####################
#
#call your definition to build the dictionary
#
#####################
print(LoadPhylip(DEFAULT_FILE_NAME))
real_sequences = LoadPhylip(DEFAULT_FILE_NAME)


{'KF927984': 'ATGAATCCAATTGTACTAACCATTATCATCTCCAGCCTCGGCCTAGGCACCACAATAACATTTATAGGCTCACATTGACTATTTATGTGAATAGGTCTCGAAATTAATACCCTAGCTATTACTCCTCTAATAATTAACCAACACCACCCACGAGCAGTAGAAGCAACCACAAAATATTTCATTACACAGGCAACCGCCTCTGCTTTACTCTTATTTGCTAGCGTAACAAACGCCTGAGCTTCTGGAGAGTGAAGTCTTCTTGAAATAATTAATCCAACATCTGCCACACTTGCAACCATTGCACTAGCCTTAAAAATTGGCCTCGCACCTTTACATTTTTGATTACCCGAAGTCCTTCAAGGACTAAATCTCACCACAGGTTTGATCTTATCAACTTGACAGAAACTCGCCCCATTTGCCATCCTTCTTCAATTATACCCCACACTAAACCCAAATCTACTATTATTTTTAGGTGTTACCTCTGTCATCGTGGGTGGTTGGGGAGGACTTAACCAAACCCAACTACGAAAAATTCTAGCTTACTCATCAATTGCCCACCTTGGTTGAATAATTATAATCCTACATTACTCCCCCAATCTTACTCAACTTAACCTAACCTTGTATATTATTATAACCTCAACAACCTTCCTACTGTTCAAAACATTTAACTCCACTAAAATTAACTCCATTTCTATCTCAGCAATTAAATCCCCACTCCTATCTATTATTACCCTGATAACCCTACTATCCCTTGGTGGCCTTCCTCCACTGACTGGATTCATACCAAAATGGTTAATTTTACAAGAACTTACTAAACAGAATTTAAATATTTCTGCTACCATTGTAGCCTTTGCAACCCTTCTCAGTCTATTCTTTTATTTACGCCTATGCTATTCCATAACCCTAACAATATCCCCAAATTCTATTCACCTATCCTCATCTTGACGAACAAAAATTCTTCAACCAAACCTTTTACTAACAACAAC

Problem 3.3. Use existing functions in this document to infer a phylogenetic tree using the real data obtained from the phylip formated file.

In [35]:
#####################
#
#use your functions to infer a phylogenetic tree given the data provided in the phylip file and print as a newick string
#
#####################

real_sequences = LoadPhylip(DEFAULT_FILE_NAME) # sets real_sequences to the matrix read by loadphylip
real_sequences = DistanceMatrix(real_sequences) # sets real_sequences to the distance matrix calculated
phylogeneticTree = ConstructPhylogeneticTree(real_sequences) # sets phylogeneticTree to the newick string calculated

print(phylogeneticTree)




(U91426,(((KF927894,JQ519166),KF927984),((JQ518749,KF92780),(((JQ519067,((JQ519064,JQ518745),JQ518747)),(JQ519066,JQ518744)),((JQ518743,(JQ51875,(KF679782,((JQ51874,JQ519168),JQ518742)))),((JQ519055,JQ518989),(((JQ519010,(JQ519048,(JQ519020,(JQ519019,((KF111729,KF928002),(JQ5190,JQ519025)))))),JQ518748),(KF927997,KF927998))))))))


Problem 3.4. The tip labels in this tree you just created are NCBI genbank identifiers.  You can visit https://www.ncbi.nlm.nih.gov/ and look up the genus and species associated with each tip label.  

The tip labeled KF679782, represents a sample from *Rhincodon typus*.  Which tip or tips are most closely related to *Rhincodon typus* in your phylogenetic tree?  Please provide your response using the genus and species associated with the genbank identifier.  

Do you believe the close association between *R. typus* and its closest neighbors in your tree represent the true evolutionary relationship of these species or an erroneus association?  Please explain your answer using information we covered in class (i.e. What are the pros and cons of using the UPGMA method).

You can use the code space below to respond, simply use "#" to prevent the interpreter from compling your answer as code.





In [ ]:
# According to our mathematical inference on the evolutionary relationsihps between these organisms, it is likely that KF679782 is closely related to JQ51874, JQ519168, and JQ518742. That is, as long as it is safe to assume the constant rates of evolution.
# We cannot say, however, that one of these species evolved from the other, nor can we state the confidence to which these three species are related.